In [1]:
import pandas as pd
import requests
import streamlit as st
from dotenv import load_dotenv
import os
#Codigo para consumir dados de uma API no caso de dados de jogos, origem API RAWG.
#Apos solicitar a chave de acesso no site da RAWG, Ela e o acesso para buscar os dados dentro de do sistema


load_dotenv() #Por seguranca da chave"key" gerei um arquivo env para o git ignorar , mais e so voce gerar a sua chave no site e subistituir o comando [os.getenv('chave')] pela sua key gerada.
key = os.getenv('chave_rawg') #Chave de acesso


#Caminho da API no caso a URL, use a funcao formart f para trazer a variavel nos {}, no caso URL + KEY
url = f'https://api.rawg.io/api/games?key={key}'


#Respota da solicitacao dos dados para a api
request = requests.get(url)


#Validador da resposta, apenas uma condicional para retorno do request com o response ! padrao resposta 200
if request.status_code == 200:
    print (f'200 == {request.status_code} - Resposta OK - Acesso concedido com sucesso!')
else:
    print(f'Houve falha na resposta erro: {request.status_code}')


#Armazena os doados da resposta 200 na variavel dados no formato de json
response = request.json()


#Traz o resultado do jason e formata em formato tabela no caso um dataframe em pandas (tabela na memoria)
req = response['results']
rawg_api = pd.json_normalize(req)

#Para trazer todas as colunas para visualizacao
pd.set_option('display.max_columns', 40)

200 == 200 - Resposta OK - Acesso concedido com sucesso!


In [2]:
#Visualizar as 5 primeiras/ultimas linhas / e verificar as informacoes de tipo de dados e colunas 
rawg_api.head(2)
#tabela.tail()
#tabela.info()
rawg_api.columns

Index(['id', 'slug', 'name', 'released', 'tba', 'background_image', 'rating',
       'rating_top', 'ratings', 'ratings_count', 'reviews_text_count', 'added',
       'metacritic', 'playtime', 'suggestions_count', 'updated', 'user_game',
       'reviews_count', 'saturated_color', 'dominant_color', 'platforms',
       'parent_platforms', 'genres', 'stores', 'clip', 'tags',
       'short_screenshots', 'added_by_status.yet', 'added_by_status.owned',
       'added_by_status.beaten', 'added_by_status.toplay',
       'added_by_status.dropped', 'added_by_status.playing', 'esrb_rating.id',
       'esrb_rating.name', 'esrb_rating.slug'],
      dtype='object')

In [3]:
#Tratamento de dados
#Selecao de apenas as colunas de uso
df = rawg_api[['id', 'slug', 'name', 'released','rating','rating_top', 'ratings', 'updated',  'platforms','parent_platforms', 'genres', 'stores','tags' ]]

In [4]:
df.head(2)

,id,slug,name,released,rating,rating_top,ratings,updated,platforms,parent_platforms,genres,stores,tags
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",2025-05-17T22:07:54,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",2025-05-18T17:12:24,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."


In [5]:
#Funcao para extrair as listas de dentro do dataframe principal a exemplo metodo explode 
def extrair_coluna(df, coluna):
    # Copiar apenas a coluna do df de origem
    df_temp = df[['id', coluna]].copy()

    # Converter strings para listas de dicionários (se necessário)
    df_temp[coluna] = df_temp[coluna].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Explodir listas metodos do pandas
    df_explodido = df_temp.explode(coluna).reset_index(drop=True)

    # Normalizar os dicionários da lista
    df_normalizado = pd.json_normalize(df_explodido[coluna])

    # Juntar os dados com o id original
    df_resultado = pd.concat([df_explodido[['id']], df_normalizado], axis=1)

    return df_resultado


In [ ]:
#Excultando as funcoes para transforma em dataframe as lista que estao dentro das colunas
ratings = extrair_coluna(rawg_api,'ratings')
platforms = extrair_coluna(rawg_api,'platforms')
genres = extrair_coluna(rawg_api,'genres')
stores = extrair_coluna(rawg_api,'stores')
tags = extrair_coluna(rawg_api,'tags')

In [9]:
#Filtro da separacao das listas do df principal
platforms = platforms[['id','released_at', 'platform.name']]
genres = genres[['id', 'name']]
stores = stores[['id','store.name']]
tags = tags[['id', 'name', 'language']]

#Df princiapal apos realizar os filtros
df = df[['id', 'slug', 'name', 'released','rating','rating_top','updated']]
#Tratamento de datas
df['released'] = pd.to_datetime(df['released'])
df['updated'] = pd.to_datetime(df['updated']).dt.date
df['updated'] = pd.to_datetime(df['updated'])